<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [7]:
using System;
using System.Collections.Generic;

public interface IManufacturerInfo
{
    string GetManufacturerInfo();
}

public interface IContactInfo
{
    string GetContactInfo();
}

public interface IStorageInfo
{
    string GetStorageInfo();
}

public class OrderLine : IManufacturerInfo, IContactInfo
{
    public int ProductId { get; private set; }
    public string ProductName { get; private set; }
    public decimal Price { get; protected set; }
    public string Manufacturer { get; private set; }
    public string Country { get; private set; }
    public string ContactEmail { get; private set; }
    public string ContactPhone { get; private set; }

    public OrderLine(int productId, string productName, decimal price, string manufacturer, string country, string contactEmail, string contactPhone)
    {
        if (productId <= 0)
            throw new ArgumentException("ProductId должен быть положительным числом.", nameof(productId));
        ProductId = productId;
        ProductName = productName;
        if (price < 0)
            throw new ArgumentException("Price не может быть отрицательным.", nameof(price));
        Price = price;
        Manufacturer = manufacturer;
        Country = country;
        ContactEmail = contactEmail;
        ContactPhone = contactPhone;
    }

    public virtual void UpdatePrice(decimal newPrice)
    {
        if (newPrice + Price < 0)
            throw new ArgumentException("Price не может быть отрицательным.", nameof(newPrice));
        Price += newPrice;
    }

    public virtual decimal CalculateTotal()
    {
        return Price;
    }

    public virtual string GetProductDetails()
    {
        return $"ID: {ProductId}, Название: {ProductName}, Цена: {Price}";
    }

    public string GetManufacturerInfo()
    {
        return $"Производитель: {Manufacturer}, Страна: {Country}";
    }

    public string GetContactInfo()
    {
        return $"Email: {ContactEmail}, Телефон: {ContactPhone}";
    }
}

public class StandardLine : OrderLine, IStorageInfo
{
    public int Units { get; private set; }
    public string StorageLocation { get; private set; }
    public string UnitType { get; private set; }

    public StandardLine(int productId, string productName, decimal price, string manufacturer, string country, string contactEmail, string contactPhone, int units, string storageLocation, string unitType)
        : base(productId, productName, price, manufacturer, country, contactEmail, contactPhone)
    {
        if (units <= 0)
            throw new ArgumentException("Units должен быть положительным числом.", nameof(units));
        Units = units;
        StorageLocation = storageLocation;
        UnitType = unitType;
    }

    public override decimal CalculateTotal()
    {
        return Price * Units;
    }

    public override string GetProductDetails()
    {
        return base.GetProductDetails() + $", Количество: {Units} {UnitType}";
    }

    public string GetStorageInfo()
    {
        return $"Место хранения: {StorageLocation}";
    }
}

public class SpecialLine : StandardLine
{
    public decimal Discount { get; private set; }
    public string DiscountReason { get; private set; }
    public string DiscountCode { get; private set; }

    public SpecialLine(int productId, string productName, decimal price, string manufacturer, string country, string contactEmail, string contactPhone, int units, string storageLocation, string unitType, decimal discount, string discountReason, string discountCode)
        : base(productId, productName, price, manufacturer, country, contactEmail, contactPhone, units, storageLocation, unitType)
    {
        if (discount < 0)
            throw new ArgumentException("Discount не может быть отрицательным.", nameof(discount));
        Discount = discount;
        DiscountReason = discountReason;
        DiscountCode = discountCode;
    }

    public override decimal CalculateTotal()
    {
        return (Price * Units) - Discount;
    }

    public override string GetProductDetails()
    {
        return base.GetProductDetails() + $", Скидка: {Discount}, Причина: {DiscountReason}, Код: {DiscountCode}";
    }

    public string GetDiscountInfo()
    {
        return $"Скидка: {Discount}, Причина: {DiscountReason}, Код: {DiscountCode}";
    }
}

public class FreeLine : SpecialLine
{
    private decimal Prepayment { get; set; }
    public string PrepaymentType { get; private set; }
    public string GiftMessage { get; private set; }

    public FreeLine(int productId, string productName, decimal price, string manufacturer, string country, string contactEmail, string contactPhone, int units, string storageLocation, string unitType, decimal discount, string discountReason, string discountCode, decimal prepayment, string prepaymentType, string giftMessage)
        : base(productId, productName, price, manufacturer, country, contactEmail, contactPhone, units, storageLocation, unitType, discount, discountReason, discountCode)
    {
        if (prepayment < 0)
            throw new ArgumentException("Prepayment не может быть отрицательным.", nameof(prepayment));
        Prepayment = prepayment;
        PrepaymentType = prepaymentType;
        GiftMessage = giftMessage;
    }

    public override decimal CalculateTotal()
    {
        return Price * Units - Discount - Prepayment;
    }

    public override string GetProductDetails()
    {
        return base.GetProductDetails() + $", Предоплата: {Prepayment} ({PrepaymentType}), Сообщение: {GiftMessage}";
    }

    public string GetPrepaymentInfo()
    {
        return $"Предоплата: {Prepayment}, Тип: {PrepaymentType}";
    }

    public string GetGiftMessage()
    {
        return $"Подарочное сообщение: {GiftMessage}";
    }
}

public class OrderCollection<T> where T : OrderLine
{
    private List<T> _orderLines = new List<T>();

    public void AddOrderLine(T orderLine)
    {
        _orderLines.Add(orderLine);
    }

    public void RemoveOrderLine(T orderLine)
    {
        _orderLines.Remove(orderLine);
    }

    public decimal CalculateTotal()
    {
        decimal total = 0;
        foreach (var line in _orderLines)
        {
            total += line.CalculateTotal();
        }
        return total;
    }

    public void PrintOrderDetails()
    {
        foreach (var line in _orderLines)
        {
            Console.WriteLine(line.GetProductDetails());
        }
    }
}



// Пример использования
OrderLine orderLine = new OrderLine(1, "Продукт", 100.0m, "ООО Рога и Копыта", "Россия", "info@roga.ru", "+7-900-000-00-00");
Console.WriteLine(orderLine.GetProductDetails());
Console.WriteLine(orderLine.GetManufacturerInfo());
Console.WriteLine(orderLine.GetContactInfo());

StandardLine standardLine = new StandardLine(2, "Стандартный продукт", 50.0m, "ООО Бумага+", "Беларусь", "contact@bumaga.by", "+375-29-111-22-33", 3, "Склад 1", "шт");
Console.WriteLine(standardLine.GetProductDetails());
Console.WriteLine(standardLine.GetManufacturerInfo());
Console.WriteLine(standardLine.GetContactInfo());
Console.WriteLine(standardLine.GetStorageInfo());

SpecialLine specialLine = new SpecialLine(3, "Специальный продукт", 80.0m, "ООО Пиши+", "Китай", "service@pishi.cn", "+86-10-1234-5678", 2, "Склад 2", "шт", 15.0m, "Акция", "DISCOUNT15");
Console.WriteLine(specialLine.GetProductDetails());
Console.WriteLine(specialLine.GetManufacturerInfo());
Console.WriteLine(specialLine.GetContactInfo());
Console.WriteLine(specialLine.GetStorageInfo());
Console.WriteLine(specialLine.GetDiscountInfo());

FreeLine freeLine = new FreeLine(4, "Бесплатный продукт", 200.0m, "ООО Подаркофф", "Германия", "gift@podarkoff.de", "+49-30-9876-5432", 1, "Склад 3", "шт", 50.0m, "Праздник", "GIFT50", 30.0m, "Бонус", "С праздником!");
Console.WriteLine(freeLine.GetProductDetails());
Console.WriteLine(freeLine.GetManufacturerInfo());
Console.WriteLine(freeLine.GetContactInfo());
Console.WriteLine(freeLine.GetStorageInfo());
Console.WriteLine(freeLine.GetDiscountInfo());
Console.WriteLine(freeLine.GetPrepaymentInfo());
Console.WriteLine(freeLine.GetGiftMessage());

OrderCollection<OrderLine> orderCollection = new OrderCollection<OrderLine>();
orderCollection.AddOrderLine(orderLine);
orderCollection.PrintOrderDetails();

OrderCollection<StandardLine> standardCollection = new OrderCollection<StandardLine>();
standardCollection.AddOrderLine(standardLine);
standardCollection.PrintOrderDetails();

OrderCollection<SpecialLine> specialCollection = new OrderCollection<SpecialLine>();
specialCollection.AddOrderLine(specialLine);
specialCollection.PrintOrderDetails();

OrderCollection<FreeLine> freeCollection = new OrderCollection<FreeLine>();
freeCollection.AddOrderLine(freeLine);
freeCollection.PrintOrderDetails();

ID: 1, Название: Продукт, Цена: 100,0
Производитель: ООО Рога и Копыта, Страна: Россия
Email: info@roga.ru, Телефон: +7-900-000-00-00
ID: 2, Название: Стандартный продукт, Цена: 50,0, Количество: 3 шт
Производитель: ООО Бумага+, Страна: Беларусь
Email: contact@bumaga.by, Телефон: +375-29-111-22-33
Место хранения: Склад 1
ID: 3, Название: Специальный продукт, Цена: 80,0, Количество: 2 шт, Скидка: 15,0, Причина: Акция, Код: DISCOUNT15
Производитель: ООО Пиши+, Страна: Китай
Email: service@pishi.cn, Телефон: +86-10-1234-5678
Место хранения: Склад 2
Скидка: 15,0, Причина: Акция, Код: DISCOUNT15
ID: 4, Название: Бесплатный продукт, Цена: 200,0, Количество: 1 шт, Скидка: 50,0, Причина: Праздник, Код: GIFT50, Предоплата: 30,0 (Бонус), Сообщение: С праздником!
Производитель: ООО Подаркофф, Страна: Германия
Email: gift@podarkoff.de, Телефон: +49-30-9876-5432
Место хранения: Склад 3
Скидка: 50,0, Причина: Праздник, Код: GIFT50
Предоплата: 30,0, Тип: Бонус
Подарочное сообщение: С праздником!
ID: 